During development of [NI Water](https://github.com/andrewbolster/bolster/pull/1009), the dataset changed underfoot and became disconnected from the [OpenDataNI dataset](https://admin.opendatani.gov.uk/dataset/ni-water-customer-tap-authorised-supply-point-results) that drove the mapping between Zones and Postcodes

Presumably, it's a load of postcodes (all BT34) that didn't participate in the 2023, but did in 2022, which doesn't make any sense either because the api call ("https://www.niwater.com/water-quality-lookup.ashx?z={zone_code}") is just returning blank. (Zone ZS0904)

Anyway, this is here just to remind me when this all goes to hell next time. 

In [1]:
from bolster.data_sources import ni_water

In [2]:
df = ni_water.get_water_quality()

In [3]:
df

,Water Supply Zone,Total Hardness (mg/l),Magnesium (mg/l),Potassium (mg/l),Calcium (mg/l),Total Hardness (mg CaCO3/l),Clark English Degrees,French Degrees,German Degrees,NI Hardness Classification,Dishwasher Setting
ZS1002,Carran Hill Camly,48.5,5.5,4.1,39.5,121.5,8.5,12.1,6.8,Slightly Hard,1 - 2
ZN0702,Glenhordial Omagh,26.3,1.8,0.9,23.3,65.8,4.6,6.6,3.7,Moderately Soft,0 - 1
ZN0804,Killyhevlin East,60.6,3.9,2.7,54.3,151.8,10.6,15.2,8.5,Moderately Hard,1 - 2
ZS0111,Dunore Point Hydepark,60.8,8.6,3.7,46.6,152.5,10.7,15.2,8.5,Moderately Hard,1 - 2
ZS0814,Castor Bay Moira,62.1,8.4,3.9,48.3,155.8,10.9,15.6,8.7,Moderately Hard,1 - 2
ZN0801,Belleek Garrison,37.4,3.3,2.8,32.3,94.2,6.6,9.4,5.3,Moderately Soft,0 - 1
ZS0103,Belfast Ballyhanwood,17.4,1.8,0.9,14.4,43.4,3.0,4.3,2.4,Soft,0
ZS0101,Dunore Ballygomartin North,60.8,8.7,3.8,46.3,152.0,10.6,15.2,8.5,Moderately Hard,1 - 2
ZS0601,Drumaroad Ballynahinch,9.5,0.6,0.4,8.5,23.8,1.7,2.4,1.3,Soft,0
ZS0502,Poleglass Dunmurry,61.1,8.4,3.9,47.4,153.6,10.8,15.4,8.6,Moderately Hard,1 - 2


In [4]:
maping = ni_water.get_postcode_to_water_supply_zone()

In [5]:
{ code for code, zone in maping.items() if zone == 'ZS0904'}

{'BT34 2LP',
 'BT34 2EW',
 'BT34 1EW',
 'BT34 3SD',
 'BT34 4TP',
 'BT34 2QQ',
 'BT34 3JH',
 'BT34 2TJ',
 'BT34 3FA',
 'BT34 3ET',
 'BT34 2AS',
 'BT34 2QZ',
 'BT34 3HS',
 'BT34 3DY',
 'BT34 3XA',
 'BT34 4BA',
 'BT34 2DD',
 'BT34 4NJ',
 'BT34 3JB',
 'BT34 4LH',
 'BT34 3EZ',
 'BT34 4LW',
 'BT34 5XD',
 'BT34 1HG',
 'BT34 2ET',
 'BT34 5QQ',
 'BT34 1LL',
 'BT34 3AP',
 'BT34 2SR',
 'BT34 3FY',
 'BT34 5AB',
 'BT34 3NY',
 'BT34 2EG',
 'BT34 4UW',
 'BT34 1BP',
 'BT34 4PJ',
 'BT34 4XU',
 'BT34 1GY',
 'BT34 1EQ',
 'BT34 5EE',
 'BT34 4PW',
 'BT34 1SF',
 'BT34 2PP',
 'BT34 1PQ',
 'BT34 4XE',
 'BT34 5QX',
 'BT34 3GF',
 'BT34 3PP',
 'BT34 4PP',
 'BT34 4SF',
 'BT34 3PW',
 'BT34 2SG',
 'BT34 4AB',
 'BT34 1RS',
 'BT34 1RE',
 'BT34 3LT',
 'BT34 2PN',
 'BT34 4PN',
 'BT34 2AQ',
 'BT34 2AR',
 'BT34 3XB',
 'BT34 5RU',
 'BT34 2GD',
 'BT34 5HH',
 'BT34 2QG',
 'BT34 1SW',
 'BT34 4UN',
 'BT34 2AW',
 'BT34 5ED',
 'BT34 5UX',
 'BT34 3XT',
 'BT34 1HP',
 'BT34 2FF',
 'BT34 4LB',
 'BT34 5DP',
 'BT34 4GE',
 'BT34 4QT',

In [6]:
data = ni_water.get_water_quality_by_zone('ZS0101')

In [7]:
data.index

Index(['Water Supply Zone', 'Total Hardness (mg/l)', 'Magnesium (mg/l)',
       'Potassium (mg/l)', 'Calcium (mg/l)', 'Total Hardness (mg CaCO3/l)',
       'Clark English Degrees', 'French Degrees', 'German Degrees',
       'NI Hardness Classification', 'Dishwasher Setting'],
      dtype='object', name=0)

In [8]:
df['NI Hardness Classification'].value_counts().keys()

CategoricalIndex(['Slightly Hard', 'Moderately Hard', 'Moderately Soft',
                  'Soft'],
                 categories=['Soft', 'Moderately Soft', 'Slightly Hard', 'Moderately Hard'], ordered=True, dtype='category', name='NI Hardness Classification')